In [1]:
# coding: utf-8

from pylab import *
import pandas as pd
import skimage
from skimage import io
from skimage import draw as dr
import os.path as osp
import os
import scipy.interpolate as sci
import sys
import pickle
# Enable inline plotting
%matplotlib inline
plt.style.use('ggplot') # Make the graphs a bit prettier
# figsize(15, 5)
pd.set_option("display.max_columns",60)
#pd.options.display.max_columns

color_list=[c['color'] for c in list(plt.rcParams['axes.prop_cycle'])]

In [2]:
def scale_dim(df,timescale,lengthscale):
    #time
    df['t']=df['frame']*timescale
        
def compute_parameters(df):
    """This function computes different parameters: velocity, ... """
    r,c=df.shape
    
    #velocity components
    for dim in ['x','y','z']:
        a=np.empty(r)
        a[:]=np.nan
        df['v'+dim]=a
        groups=df.groupby(['traj'])
        for traj in df['traj'].unique():
            traj_group = groups.get_group(traj)
            components=(traj_group[dim].shift(-1)-traj_group[dim])/(traj_group['t'].shift(-1)-traj_group['t'])
            ind=traj_group.index.values
            df.loc[ind[1:],'v'+dim]=components[:-1].values
    #velocity modulus
    df['v']=sqrt(df['vx']**2+df['vy']**2+df['vz']**2)
    
def get_info(dirdata):
    filename=osp.join(dirdata,"info.txt")
    if osp.exists(filename):
        with open(filename) as f:
            info={'lengthscale':-1,'delta_t':-1,'columns':-1}
            for line in f:        
                if ('lengthscale' in line)==True:
                    if len(line.split())==3:
                        info['lengthscale']= float(line.split()[2])
                elif ('delta_t' in line)==True:
                    if len(line.split())==3:
                        info['delta_t'] = float(line.split()[2])
                elif ('columns' in line)==True:
                    if len(line.split())==3:
                        info['columns'] = line.split()[2].split(',')
    else: 
        print "ERROR: info.txt doesn't exist or is not at the right place"
    return info

def plot_frame_cells(df,groups,frame,data_dir,plot_traj=False,filtered_tracks=None):
    """ Print the tracked pictures with updated (=relinked) tracks"""
    print '\rplotting frame '+str(frame),
    sys.stdout.flush()
    if filtered_tracks is None:
        track_dir=osp.join(data_dir,'traj')
    else:
        track_dir=osp.join(data_dir,'traj_subset')
    if osp.isdir(track_dir)==False:
        os.mkdir(track_dir)
    group=groups.get_group(frame).reset_index(drop=True)
    r,c=group.shape
    if plot_traj:
        track_groups=df.groupby(['traj'])
    #import image
    filename=osp.join(data_dir,'raw/max_intensity_%04d.png'%int(frame))
    im = io.imread(filename)
    n,m,d = im.shape
    fig=figure(frameon=False)
    fig.set_size_inches(m/300,n/300)
    ax = fig.add_axes([0, 0, 1, 1])
    ax.imshow(im,aspect='auto')
    xmin, ymin, xmax, ymax=ax.axis('off')
    if filtered_tracks is None:
        for i in range(0,r):
            #write label
            x=group.loc[i,'x']
            y=group.loc[i,'y']
            s='%d'%(group.loc[i,'traj'])
            ax.text(x,y,s,fontsize=5,color='w')
            if plot_traj:
                #plot trajectory
                traj=get_obj_traj(track_groups,s,max_frame=frame)
                traj_length,c=traj.shape
                if traj_length>1:
                    ax.plot(traj['x'],traj['y'],ls='-',color=color_list[i%7])
                    ax.axis([xmin, ymin, xmax, ymax])

    else:
        for i,track in enumerate(filtered_tracks):
            if (group['traj']==track).any():
                #write label
                x=group[group['traj']==track]['x'].values[0]
                y=group[group['traj']==track]['y'].values[0]
                s='%d'%track
                ax.text(x,y,s,fontsize=5,color='w')
                if plot_traj:
                    #plot trajectory
                    traj=get_obj_traj(track_groups,track,max_frame=frame)
                    traj_length,c=traj.shape
                    if traj_length>1:
                        ax.plot(traj['x'],traj['y'],ls='-',color=color_list[i%7])
                        ax.axis([xmin, ymin, xmax, ymax])
    filename=osp.join(track_dir,'traj_%04d.png'%int(frame))
    fig.savefig(filename, dpi=300)
    close('all')
                      
def get_obj_traj(track_groups,track,max_frame=None):
    '''gets the trajectory of an object. track_groups is the output of a groupby(['relabel'])'''
    group=track_groups.get_group(track)
    trajectory=group[['frame','t','x','y','v']].copy()
    if max_frame is not None:
        trajectory=trajectory[trajectory['frame']<=max_frame]
    return trajectory.reset_index(drop=True)

def plot_frame_vfield(df,groups,frame,data_dir,interpolate=False,filtered_tracks=False,grid_step=10):
    """ Print the tracked pictures with updated (=relinked) tracks"""
    print '\rplotting frame '+str(frame),
    sys.stdout.flush()
    if filtered_tracks:
        track_dir=osp.join(data_dir,'vfield_subset')
    else:
        track_dir=osp.join(data_dir,'vfield')
        
    if osp.isdir(track_dir)==False:
        os.mkdir(track_dir)
    group=groups.get_group(frame).reset_index(drop=True)
    r,c=group.shape
    #import image
    filename=osp.join(data_dir,'raw/max_intensity_%04d.png'%int(frame))
    im = io.imread(filename)
    n,m,d = im.shape
    fig=figure(frameon=False)
    fig.set_size_inches(m/300,n/300)
    ax = fig.add_axes([0, 0, 1, 1])
    ax.imshow(im,aspect='auto')
    xmin, ymin, xmax, ymax=ax.axis('off')
    if interpolate:
        grid_x, grid_y = np.mgrid[0:m:m/grid_step, 0:n:n/grid_step]
        grid=[]
        for p in ['vx','vy','vz']:
            points=df[['x','y']].values
            values=df[p].values
            grid.append(sci.griddata(points, values, (grid_x, grid_y), method='linear'))
        quiver(grid_x,grid_y,grid[0],grid[1],grid[2])
        ax.axis([xmin, ymin, xmax, ymax])
    else:
        quiver(group['x'],group['y'],group['vx'],group['vy'],group['vz'])
        ax.axis([xmin, ymin, xmax, ymax])
    filename=osp.join(track_dir,'vfield_%04d.png'%int(frame))
    fig.savefig(filename, dpi=300)
    close('all')


In [3]:
def run(data_dir,refresh=False,plot_frame=True,plot_data=True,plot_modified_tracks=False,plot_all_traj=False):
    #import
    pickle_fn=osp.join(data_dir,"data_base.p")
    if osp.exists(pickle_fn)==False or refresh:
        data=loadtxt(osp.join(data_dir,'table.txt'))
        info=get_info(data_dir)
        for inf in ['lengthscale','delta_t','columns']:
            if info[inf]==-1:
                print "WARNING: "+inf+" not provided in info.txt"
        lengthscale=info["lengthscale"];timescale=info["delta_t"];columns=info["columns"]
        df=pd.DataFrame(data[:,1:],columns=columns) 
        #scale data
        scale_dim(df,timescale,lengthscale)
        compute_parameters(df)
        #update pickle
        pickle.dump(df, open( osp.join(data_dir,"data_base.p"), "wb" ) )
    else:
        df=pickle.load( open( pickle_fn, "rb" ))
    
    return df

#     #####################################
#     ######  Plotting starts here  #######
#     #####################################

#     if plot_frame:
#         groups = df.groupby(["ImageNumber"])
#         if plot_all_traj:
#             for frame in range(framemin,framemax+1):
#                 plot_frame_obj(df,groups,frame,data_dir,filtered_tracks=df['relabel'].values.tolist(),plot_traj=True)
#         else:
#             for frame in range(framemin,framemax+1):
#                 plot_frame_obj(df,groups,frame,data_dir)

#     #plot measurements
#     if plot_data:
#         print 'plotting data...'
#         tracks_to_plot=select_tracks(df,'long_track',min_track_length,timescale)
#         for param_y in param_y_list:
#             plot_params(df,tracks_to_plot,param_y,data_dir,param_x='Metadata_frame')
#         plot_hist_avg(df,tracks_to_plot,param_y_list,data_dir)

In [4]:
data_dir='/home/amichaut/Desktop/charlene'
df=run(data_dir)


In [7]:
groups=df.groupby('frame')
for frame in df['frame'].unique():
    plot_frame_vfield(df,groups,frame,data_dir,interpolate=False)

plotting frame 0.0

/usr/local/lib/python2.7/dist-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/usr/local/lib/python2.7/dist-packages/numpy/ma/core.py:4139: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


plotting frame 24.0


In [5]:
tracks=df.groupby('traj')
long_tracks=[]
for t in df['traj'].unique():
    track=tracks.get_group(t)
    if track.shape[0]==25:
        long_tracks.append(t)
print len(long_tracks)
groups=df.groupby('frame')
for frame in df['frame'].unique():
    plot_frame_cells(df,groups,frame,data_dir,plot_traj=True,filtered_tracks=long_tracks)


493
plotting frame 24.0


In [65]:
tracks=df.groupby('traj')
t=pd.concat([tracks.get_group(1),tracks.get_group(5)])

In [5]:
df

,traj,frame,x,y,z,m0,m1,m2,m3,m4,NPscore,t,vx,vy,vz,v
0,1.0,0.0,7.768,217.405,0.943,0.943,2.296,6.382,20.352,72.011,4189.398,0.0,NaN,NaN,NaN,NaN
1,1.0,1.0,7.371,217.455,1.073,0.951,2.414,7.008,23.083,83.315,3995.026,8.0,-0.049625,0.006250,0.016250,0.052591
2,1.0,2.0,7.753,217.944,1.050,0.991,2.521,7.477,24.807,89.354,3938.991,16.0,0.047750,0.061125,-0.002875,0.077618
3,1.0,3.0,7.806,217.595,0.896,0.879,2.284,6.152,18.659,62.139,4144.341,24.0,0.006625,-0.043625,-0.019250,0.048141
4,1.0,4.0,7.699,218.308,1.039,1.005,2.489,7.341,24.317,87.649,3856.469,32.0,-0.013375,0.089125,0.017875,0.091879
5,1.0,5.0,8.371,217.901,1.028,0.796,2.271,6.217,19.497,67.864,4270.144,40.0,0.084000,-0.050875,-0.001375,0.098215
6,1.0,6.0,7.665,217.050,1.046,1.023,2.447,7.018,22.467,78.151,3716.076,48.0,-0.088250,-0.106375,0.002250,0.138234
7,1.0,7.0,7.271,218.948,1.748,1.523,2.730,8.410,28.167,100.488,2477.484,56.0,-0.049250,0.237250,0.087750,0.257708
8,1.0,8.0,7.915,216.977,0.846,0.809,2.495,7.461,25.291,93.793,2816.724,64.0,0.080500,-0.246375,-0.112750,0.282654
9,1.0,9.0,7.453,217.403,1.161,0.997,2.378,6.807,22.250,80.272,4332.815,72.0,-0.057750,0.053250,0.039375,0.087869
